In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import copy
from typing import List, Tuple

from cartpole_dynamics import CartpoleDynamics
from dynamics import *
from quadratic_program import QuadraticProgram
from solvers import InfeasibleStartNewton, FeasibleStartNewton
from trajectory_optimizer import TrajectoryOptimizer

In [ ]:
np.set_printoptions(threshold=np.inf)

## References

https://cse.lab.imtlucca.it/~bemporad/publications/papers/ijc_rtiltv.pdf

https://web.stanford.edu/~boyd/papers/pdf/fast_mpc.pdf

https://www.imtek.de/professuren/systemtheorie/events/dateien/directshootingmethods

https://www.matthewpeterkelly.com/tutorials/trajectoryOptimization/cartPoleCollocation.svg#frame1114

In [ ]:
def pendulum_position(z: np.ndarray, l_pend: float):
   '''
   Returns the position of the mass at the end of a pendulum attached to a cart given some current system state.
   - z = Cartpole system state: [cart position, pendulum angle, cart speed, pendulum angular speed]
   - l_pend = Length of a pendulum
   '''
   return np.array([z[0] + l_pend * np.sin(z[1]), -l_pend * np.cos(z[1])])

In [ ]:
def plot_pendulum_time_sequence(states: List[np.ndarray], l_pend: float, keyframe_stride: int=4, filename: str=None):
   '''
   Plots the cart position, pendulum arm, and pendulum mass position for a sequence of 4d
   states indicating the system's [cart_pos, pendulum_angle, cart_speed, pendulum_angular_speed].

   Add shapes to matplotlib plot:
   https://www.scaler.com/topics/matplotlib/plot-shape-matplotlib/

   - states = List of cartpole states
   - l_pend = Length of the pendulum
   - keyframe_stride = Number of frames to skip plotting the whole cartpole system
   - filename = Name of the file to save the plot to
   '''

   positions = [pendulum_position(z, l_pend) for z in states]

   keyframe_states = [s for i, s in enumerate(states) if (i % keyframe_stride) == 0]

   keyframe_colors = cm.rainbow(np.linspace(0, 1, int(len(states) / keyframe_stride) + 1))
   colors = cm.rainbow(np.linspace(0, 1, len(states)))
   fig = plt.figure()
   ax = fig.add_subplot()
   # ax.set_ylim([-1.25 * l_pend, 1.25 * l_pend])
   for i, ks in enumerate(keyframe_states):
      cart = plt.Rectangle((ks[0] - 0.125/2, 0.0), 0.125, 0.125/2, fc=keyframe_colors[i])
      pend_pos = pendulum_position(ks, l_pend)
      plt.plot([ks[0], pend_pos[0]], [0.0, pend_pos[1]], color=keyframe_colors[i])
      plt.gca().add_patch(cart)
   plt.scatter([p[0] for p in positions], [p[1] for p in positions], color=colors, s=0.25)
   
   # plt.axis('equal')
   plt.ylim(-1.25 * l_pend, 1.25 * l_pend)
   plt.gca().set_aspect('equal')
   if filename is not None:
      plt.savefig(filename, dpi=8000.0, bbox_inches='tight')

   fig = plt.figure()
   plt.title("pendulum angle")
   plt.plot([z[1] for z in states])
   fig = plt.figure()
   plt.title("position")
   plt.plot([z[0] for z in states])
   fig = plt.figure()
   plt.title("cart speed")
   plt.plot([z[2] for z in states])
   fig = plt.figure()
   plt.title("pendulum angular speed")
   plt.plot([z[3] for z in states])

   plt.show()

In [ ]:
def cartpole_demo(z_init: np.ndarray, u_init: float, t_max: float, u_max: float=1.0):
   zs = []
   zs.append(copy.copy(z_init))
   dt = 0.01

   cp_dynamics = CartpoleDynamics(1.0, 1.0, 1.0, -u_max, u_max)

   for i in range(int(t_max / dt)):
      zs.append(cp_dynamics.step(zs[-1], u_init, dt, dt * i))

   plot_pendulum_time_sequence(zs, 1.0)

In [ ]:
cartpole_demo(np.array([0.0, 0.0, 0.0, 0.0]), 10.0, 2, u_max=10.0)

The dynamics of a system are governed by

$\dot{\mathbf{z}} = f(t, \mathbf{z}, u)$

where $t$ is time, $\mathbf{z}$ is the system's state, and $u$ is a control input applied to the system. Then the system's linearized dynamics are given by

$\dot{\mathbf{z}} = f(t, \mathbf{z}, u) \approx f(t_0, \mathbf{z}_0, u_0) + \nabla_{\mathbf{z}}f(t_0, \mathbf{z}_0, u_0)^T (\mathbf{z} - \mathbf{z}_0) + \nabla_uf(t_0, \mathbf{z}_0, u_0)^T(u - u_0)$

or, using the terms from the following function,

$\dot{\mathbf{z}} = \mathbf{r}(t_0, \mathbf{z}_0, u_0) + A \mathbf{z} + \mathbf{b}u$

for some initial time, state, and control input.

\begin{equation}
x(t) = e^{A(t - t_0)}x(t_0) + e^{At}\int_{t_0}^{t}e^{A\tau} \left( b u(\tau) + f \right) d\tau
\end{equation}

When $u(\tau) = u_0$ for $t_0 \leq t \leq t$ the convolution integral becomes

\begin{equation}
x(t) = e^{A(t - t_0)}x(t_0) + \left( e^{A(t - t_0)} - \mathbb{I} \right)A^{-1}(b u_0 + f)
\end{equation}

$A \in \mathbb{R}^{n \times n}$ is the Jacobian of the nonlinear system dynamics with respect to $x \in \mathbb{R}^n$

$b \in \mathbb{R}^n$ is the Jacobian of the nonlinear system dynamics with respect to $u \in \mathbb{R}$

$f \in \mathbb{R}^n$ is the constant term that comes from linearizing the system dynamics about a non-fixed point

The state transition matrix is defined as

\begin{equation}
\Phi(t) = e^{At}
\end{equation}

and transitioning from one state to the next state is done via

\begin{equation}
\mathbf{x}_{t+\delta t} = \Phi(t + \delta t, t) \mathbf{x}_{t}
\end{equation}

In [ ]:
def plot_time_sequence(positions: List[np.ndarray]):
   '''
   Plots a sequence of 2d positions in the xy plane, where x and y indicate the position of
   *something* that has evolved over time.

   - positions = List of 2d positions
   '''
   colors = cm.rainbow(np.linspace(0, 1, len(positions)))
   fig = plt.figure()
   ax = fig.add_subplot()
   plt.scatter([p[0] for p in positions], [p[1] for p in positions], color=colors)
   ax.axis('equal')
   plt.show()

In [ ]:
def plot_control_input(us: List[np.ndarray], dt, filename: str = None):
   '''
   Plots a sequence of control inputs

   - us = A sequence of control inputs
   - dt = Time duration over which the LTI system is simulated
   - filename = Name of the file the plot should be saved to
   '''

   t_vals = [dt * n for n in range(len(us))]
   plt.plot(t_vals, [u for u in us])
   # plt.plot([u for u in us])
   plt.xlabel("Time (s)")
   plt.ylabel("Control effort")
   plt.show()
   if filename is not None:
      plt.savefig(filename, dpi=800.0, bbox_inches='tight')


## Trajectory Optimization

In [ ]:
def cartpole_trajectory_optimizer_with_class(
   N: int,
   T: float,
   dynamics: CartpoleDynamics,
   z_0: np.ndarray,
   z_f: np.ndarray | None=None,
   z_f_penalty: float | None=None,
   ref_trajectory: List[np.ndarray] | None=None,
   ref_trajectory_weight: float | None=None,
   plot_stuff: bool=True,
   filename: str | None=None
):
   effort_weights = np.ones(N * dynamics.control_size)

   state_weights = None
   if ref_trajectory is not None:
      if ref_trajectory_weight is None:
         state_weights = 100.0 * np.tile([1.0, 1.0, 0.0, 0.0], N)
      else:
         state_weights = ref_trajectory_weight * np.tile([1.0, 1.0, 0.0, 0.0], N)

   to = TrajectoryOptimizer(
      effort_weights,
      N,
      T,
      z_0,
      dynamics,
      final_state=z_f,
      reference_trajectory=ref_trajectory,
      state_weights=state_weights,
      final_state_penalty=z_f_penalty
   )

   # zs for warm-starting infeasible newton
   delta_tau = (1.0 / N)
   warm_zs = None
   if z_f is not None and z_f_penalty is None:
      warm_zs = [(1 - i * delta_tau) * z_0 + i * delta_tau * z_f for i in range(N)]
   else:
      warm_zs = [(1 - i * delta_tau) * z_0 + i * delta_tau * np.zeros(dynamics.state_size) for i in range(N)]
   warm_us = [0.000,] * (N)

   print("z_0: ", warm_zs[0])
   print("z_f: ", warm_zs[-1])

   x_init = np.hstack(warm_us + warm_zs)
   v_init = None
   if z_f is not None and z_f_penalty is None:
      v_init = np.ones(dynamics.state_size * (N + 1)) * 0.01
   else:
      v_init = np.ones(dynamics.state_size * N) * 0.01

   x_out, v_out, feasible = to.gogogo(x_init, v_init, max_num_newton_iters=100)

   if not feasible:
      print("problem isn't feasible")
      return

   dense_zs = to.get_states(x_out)
   dense_zs.insert(0, z_0)
   dense_us = to.get_inputs(x_out)

   print("last control input:", dense_us[-1])

   if plot_stuff:

      control_input_filename = None
      trajectory_filename = None
      if filename is not None:
         z_0_str = "_".join([str(v) for v in z_0.tolist()])
         if z_f is not None:
            z_f_str = "_".join([str(v) for v in z_f.tolist()])
         else:
            z_f_str = "_".join([str(v) for v in np.zeros(dynamics.state_size).tolist()])
         config_str = f'_T={T}_N={N}_z0={z_0_str}_zf={z_f_str}_'
         control_input_filename = filename + config_str + "_control_inputty.png"
         trajectory_filename = filename + config_str + "_cartpole_swingy.png"
      
      plot_pendulum_time_sequence(dense_zs, dynamics._l_pend, 2, trajectory_filename)
      plot_control_input(dense_us, T / N, control_input_filename)

## Modifiable parameters

Some configurations might make the problem infeasible.

In [ ]:
# Number of timesteps
N = 128
# Total amount of time
T = 1.0

u_min = -15.0
u_max = 15.0

m_pend = 0.3
m_cart = 0.3
l_pend = 0.5

dynamics = CartpoleDynamics(l_pend, m_pend, m_cart, u_min, u_max)

z_0 = np.array([0.0, 0.0, 0.0, 0.0])
z_final = np.array([1.0, np.pi, 0.0, 0.0])

# control_weighting = np.ones(N * dynamics.control_size)
# control_weighting = np.ones((N + 1) * dynamics.control_size)

In [ ]:
# ref_traj = []
# for i in range(N):
#    ref_traj.append(np.array([0.0, i * 3.0 / N, 0.0, 0.0]))
# cartpole_trajectory_optimizer_with_class(N, T, dynamics, z_0, z_f=z_final, ref_trajectory=ref_traj)

# ref_traj = [z_final,] * N
# cartpole_trajectory_optimizer_with_class(N, T, dynamics, z_0, ref_trajectory=ref_traj)

cartpole_trajectory_optimizer_with_class(N, T, dynamics, z_0, z_f=z_final, z_f_penalty=100000.0)

In [ ]:
cartpole_trajectory_optimizer_without_class(N, T, dynamics, z_0, z_final)